In [1]:
COLORS = {
    'reset': '\033[0m',

    'red': '\033[31m',
    'green': '\033[32m',
    'yellow': '\033[33m',
}

def GetColor(color_name: str) -> str:
    return COLORS.get(color_name, COLORS['reset'])

def Colored(text: str, color_name: str = 'green') -> str:
    return f"{GetColor(color_name)}{text}{GetColor('reset')}"


In [ ]:
import numpy as np

def Round(number: float, precision: int = 4) -> float:
    factor = 10 ** precision
    return round(float(number) * factor) / factor

def MakeVectors(*args) -> np.ndarray:
    return np.array(object=args)

def DotProduct(matrix: np.ndarray, vector: np.ndarray) -> np.ndarray:
    return np.dot(matrix, vector)

# =============================================================================

def TransitionMatrix(matrix: list) -> np.ndarray:
    return MakeVectors(*matrix)

def PeopleVector(people_industry: int, people_academia: int) -> np.ndarray:
    return MakeVectors(people_industry, people_academia)

# =============================================================================

def CalculatePeopleDistribution(percent: float, total_people: int) -> int:
    return percent * total_people, (1 - percent) * total_people

def CalculatePeopleInitialState(
    total_people: int, percentage_industry: float = None,  percentage_academia: float = None, 
) -> np.ndarray:

    if percentage_industry is not None:
        people_industry, people_academia = CalculatePeopleDistribution(percentage_industry, total_people)
        return PeopleVector(people_industry, people_academia)

    elif percentage_academia is not None:
        people_academia, people_industry = CalculatePeopleDistribution(percentage_academia, total_people)
        return PeopleVector(people_industry, people_academia)
    
    else:
        raise ValueError("Either percentage_industry or percentage_academia must be provided.")

def CalculatePeopleNextYear(transition_matrix: np.ndarray, people: np.ndarray) -> np.ndarray:
    return DotProduct(transition_matrix, people)

# =============================================================================

def SimulateYears(
    initial_year: int, period_of_time: int,
    people: np.ndarray, transition_matrix: np.ndarray, 
    total_people: int, percentage_industry: float,
    step: int = 1,
) -> np.ndarray:

    history = []
    
    start = initial_year
    stop = initial_year + period_of_time
    
    # Store initial state with parameters
    history.append([percentage_industry, total_people, start, people[0], people[1]])

    for year in range(start + 1, stop + 1, step):
        people = CalculatePeopleNextYear(transition_matrix, people)
        # Append all parameters and calculated values for each year
        history.append([percentage_industry, total_people, year, people[0], people[1]])

    return np.array(history)

#-----------------------------------------------------------------------------

def Simulation(
    total_people: int,
    percentage_people_industry: float,
    initial_year: int,
    period_of_time: int,
    transition_matrix: np.ndarray,
    qnt_sample_total_people: int = 5,
    step_sample_total_people: int = 10_000,
    qnt_sample_percentage_people_industry: int = 5,
    step_sample_percentage_people_industry: float = 0.1,
) -> np.ndarray:
    
    all_results = []

    for i in range(qnt_sample_total_people):
        total_people_i = total_people + i * step_sample_total_people

        for j in range(qnt_sample_percentage_people_industry):
            percentage_people_industry_j = percentage_people_industry + j * step_sample_percentage_people_industry
            
            people = CalculatePeopleInitialState(
                total_people=total_people_i, 
                percentage_industry=percentage_people_industry_j,
            )

            peopleHistory = SimulateYears(
                people=people,
                initial_year=initial_year,
                period_of_time=period_of_time,
                transition_matrix=transition_matrix,
                total_people=total_people_i, # Pass total_people here
                percentage_industry=percentage_people_industry_j, # Pass percentage here
            )

            all_results.append(peopleHistory)

    # Use np.concatenate to join the smaller arrays into a single large one
    return np.concatenate(all_results, axis=0, dtype=object)

# =============================================================================

def UnpackData(history: np.ndarray) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    X_axis = history[:, 0]
    Y_axis_industry = history[:, 1]
    Y_axis_academia = history[:, 2]
    return X_axis, Y_axis_industry, Y_axis_academia

# =============================================================================

def FindChangeYears(people_history: list[tuple[int, np.ndarray]]) -> list[int]:

    # Unpack the data into separate arrays for years, industry counts, and academia counts:
    X_axis, Y_axis_industry, Y_axis_academia = UnpackData(people_history)

    # Determine the leader for each year. 
    # 1 for industry, -1 for academia, 0 for equal.
    leader_array = np.sign(Y_axis_industry - Y_axis_academia)
    
    # Find the indices where the leader changes.
    # We compare each element to the previous one.
    change_indices = np.where(leader_array[:-1] != leader_array[1:])[0]
    
    # The change happens at the *next* year, so we add 1 to the indices.
    # We then use these indices to get the corresponding years.
    change_years = X_axis[change_indices + 1]
    
    return change_years.tolist()



In [68]:
T = [
    [0.99, 0.2], 
    [0.01, 0.8],
]

transitionMatrix = TransitionMatrix(matrix=T)

In [69]:
totalPeople = 1_000
percentagePeopleIndustry = 0.1

initialYear = 2025
periodOfTime = 100
finalYear = initialYear + periodOfTime



In [70]:

Responses = Simulation(
    total_people = totalPeople,
    initial_year = initialYear,
    period_of_time = periodOfTime,
    transition_matrix = transitionMatrix,
    percentage_people_industry = percentagePeopleIndustry,
)

Responses

array([[0.1, 1000.0, 2025.0, 100.0, 900.0],
       [0.1, 1000.0, 2026.0, 279.0, 721.0],
       [0.1, 1000.0, 2027.0, 420.40999999999997, 579.59],
       ...,
       [0.5, 41000.0, 2123.0, 39047.61904589821, 1952.3809541018038],
       [0.5, 41000.0, 2124.0, 39047.619046259584, 1952.3809537404252],
       [0.5, 41000.0, 2125.0, 39047.61904654507, 1952.3809534549362]],
      shape=(2525, 5), dtype=object)

## `Option 02`:

In [77]:
def SimulateYears(
    initial_year: int, period_of_time: int,
    people: np.ndarray, transition_matrix: np.ndarray, 
    total_people: int, percentage_industry: float,
    step: int = 1,
) -> np.ndarray:

    history = []
    
    start = initial_year
    stop = initial_year + period_of_time
    
    # Store initial state with parameters
    history.append([Round(percentage_industry), total_people, start, Round(people[0]), Round(people[1])])

    for year in range(start + 1, stop + 1, step):
        people = CalculatePeopleNextYear(transition_matrix, people)
        # Append all parameters and calculated values for each year
        history.append([Round(percentage_industry), total_people, year, Round(people[0]), Round(people[1])])

    return np.array(history, dtype=object)


In [78]:
def Simulation(
    total_people: int,
    percentage_people_industry: float,
    initial_year: int,
    period_of_time: int,
    transition_matrix: np.ndarray,

    qnt_sample_total_people: int = 5,
    step_sample_total_people: int = 10_000,

    qnt_sample_percentage_people_industry: int = 5,
    step_sample_percentage_people_industry: float = 0.1,
) -> np.ndarray:
    
    all_results = []

    for i in range(qnt_sample_total_people):
        total_people_i = total_people + i * step_sample_total_people

        for j in range(qnt_sample_percentage_people_industry):
            percentage_people_industry_j = percentage_people_industry + j * step_sample_percentage_people_industry
            
            people = CalculatePeopleInitialState(
                total_people=total_people_i, 
                percentage_industry=percentage_people_industry_j,
            )

            peopleHistory = SimulateYears(
                people=people,
                initial_year=initial_year,
                period_of_time=period_of_time,
                transition_matrix=transition_matrix,
                
                total_people=total_people_i, # Pass total_people here
                percentage_industry=percentage_people_industry_j, # Pass percentage here
            )

            all_results.append(peopleHistory)

    return all_results




In [80]:

Responses = Simulation(
    total_people = totalPeople,
    initial_year = initialYear,
    period_of_time = periodOfTime,
    transition_matrix = transitionMatrix,
    percentage_people_industry = percentagePeopleIndustry,
)

Responses[10]

array([[0.1, 21000, 2025, 2100.0, 18900.0],
       [0.1, 21000, 2026, 5859.0, 15141.0],
       [0.1, 21000, 2027, 8828.61, 12171.39],
       [0.1, 21000, 2028, 11174.6019, 9825.3981],
       [0.1, 21000, 2029, 13027.9355, 7972.0645],
       [0.1, 21000, 2030, 14492.069, 6507.931],
       [0.1, 21000, 2031, 15648.7345, 5351.2655],
       [0.1, 21000, 2032, 16562.5003, 4437.4997],
       [0.1, 21000, 2033, 17284.3752, 3715.6248],
       [0.1, 21000, 2034, 17854.6564, 3145.3436],
       [0.1, 21000, 2035, 18305.1786, 2694.8214],
       [0.1, 21000, 2036, 18661.0911, 2338.9089],
       [0.1, 21000, 2037, 18942.262, 2057.738],
       [0.1, 21000, 2038, 19164.3869, 1835.6131],
       [0.1, 21000, 2039, 19339.8657, 1660.1343],
       [0.1, 21000, 2040, 19478.4939, 1521.5061],
       [0.1, 21000, 2041, 19588.0102, 1411.9898],
       [0.1, 21000, 2042, 19674.528, 1325.472],
       [0.1, 21000, 2043, 19742.8771, 1257.1229],
       [0.1, 21000, 2044, 19796.8729, 1203.1271],
       [0.1, 21000, 20